In [ ]:
# strQgen.ipynb
%run ./querygenai.ipynb

def classify_intent(query: str) -> str:
    intent_keywords = {
        "eligibility_check": ["eligible", "eligibility", "can i", "still covered"],
        "claim_status": ["claim status", "track claim", "claim update"],
        "coverage_check": ["covered", "coverage", "what is included"],
        "renewal": ["renew", "extension", "renewal"],
        "premium_info": ["premium", "cost", "price", "installment"],
        "document_requirement": ["documents", "papers", "required for"],
    }

    query_lower = query.lower()
    for intent, keywords in intent_keywords.items():
        if any(kw in query_lower for kw in keywords):
            return intent
    return "general_inquiry"

In [2]:
def extract_structured_entities(query: str) -> dict:
    import re

    entities = {}

    # Incident or procedure keywords
    if "accident" in query.lower():
        entities["incident"] = "accident"
    elif "surgery" in query.lower():
        entities["incident"] = "surgery"

    # Time-related info (basic pattern)
    time_match = re.search(r"(last month|[0-9]+\s+(days?|months?|years?)\s+ago)", query.lower())
    if time_match:
        entities["incident_time"] = time_match.group(1)

    # Policy status
    if "new policy" in query.lower():
        entities["policy_status"] = "new"
    elif "old policy" in query.lower() or "existing policy" in query.lower():
        entities["policy_status"] = "existing"
    
    return entities

In [3]:
def build_structured_query(info: dict, rewritten_query: str, raw_query: str) -> dict:
    structured = {
        "original_query": raw_query,
        "rewritten_query": rewritten_query,
        "intent": classify_intent(raw_query),
        "subject": info.get("subject"),
        "age": info.get("age"),
        "gender": info.get("gender"),
        "procedure": info.get("procedure"),
        "location": info.get("location"),
        "policy_duration": info.get("policy_duration"),
        "extracted_entities": extract_structured_entities(raw_query)
    }
    return structured

In [4]:
def compute_completeness_score(structured_query: dict) -> float:
    fields = ["subject", "age", "gender", "procedure", "location", "policy_duration"]
    filled = sum(1 for field in fields if structured_query.get(field))
    score = filled / len(fields)
    return round(score, 2)

In [5]:
query = "bike accident last month… still eligible for vehicle insurance?"
info = extract_query_info_llm(query)
rewritten = rewrite_query(info, query)
structured = build_structured_query(info, rewritten, query)

print(structured)
print("Completeness Score:", compute_completeness_score(structured))

{'original_query': 'bike accident last month… still eligible for vehicle insurance?', 'rewritten_query': 'Can the vehicle be insured? Based on the context: bike accident last month… still eligible for vehicle insurance. Eligibility may depend on vehicle condition, registration location, and past claims history. Check coverage for accidents, third-party liability, own damage, and add-ons like zero depreciation.', 'intent': 'eligibility_check', 'subject': 'vehicle insurance', 'age': None, 'gender': None, 'procedure': None, 'location': None, 'policy_duration': None, 'extracted_entities': {'incident': 'accident', 'incident_time': 'last month'}}
Completeness Score: 0.17
